Before running this, please make sure to activate and instantiate the
tutorial-specific package environment, using this
[`Project.toml`](https://raw.githubusercontent.com/juliaai/DataScienceTutorials.jl/gh-pages/__generated/isl/lab-8/Project.toml) and
[this `Manifest.toml`](https://raw.githubusercontent.com/juliaai/DataScienceTutorials.jl/gh-pages/__generated/isl/lab-8/Manifest.toml), or by following
[these](https://juliaai.github.io/DataScienceTutorials.jl/#learning_by_doing) detailed instructions.

@@dropdown
## Getting started
@@
@@dropdown-content

In [ ]:
using MLJ
import RDatasets: dataset
using PrettyPrinting
import DataFrames: DataFrame, select, Not
DTC = @load DecisionTreeClassifier pkg=DecisionTree

carseats = dataset("ISLR", "Carseats")

first(carseats, 3) |> pretty

We encode a new variable `High` based on whether the sales are higher or lower than 8 and add that column to the dataframe:

In [ ]:
High = ifelse.(carseats.Sales .<= 8, "No", "Yes") |> categorical;
carseats[!, :High] = High;

Let's now train a basic decision tree classifier for `High` given the other features after one-hot-encoding the categorical features:

In [ ]:
X = select(carseats, Not([:Sales, :High]))
y = carseats.High;

@@dropdown
### Decision Tree Classifier
@@
@@dropdown-content

In [ ]:
HotTreeClf = OneHotEncoder() |> DTC()

mdl = HotTreeClf
mach = machine(mdl, X, y)
fit!(mach);

Note `|>` is syntactic sugar for creating a `Pipeline` model from component model instances or model types.
Note also that the machine `mach` is trained on the whole data.

In [ ]:
ypred = predict_mode(mach, X)
misclassification_rate(ypred, y)

That's right... it gets it perfectly; this tends to be classic behaviour for a DTC to overfit the data it's trained on.
Let's see if it generalises:

In [ ]:
train, test = partition(eachindex(y), 0.5, shuffle=true, rng=333)
fit!(mach, rows=train)
ypred = predict_mode(mach, rows=test)
misclassification_rate(ypred, y[test])

Not really...

‎
@@
@@dropdown
### Tuning a DTC
@@
@@dropdown-content

Let's try to do a bit of tuning

In [ ]:
r_mpi = range(mdl, :(decision_tree_classifier.max_depth), lower=1, upper=10)
r_msl = range(mdl, :(decision_tree_classifier.min_samples_leaf), lower=1, upper=50)

tm = TunedModel(
    model=mdl,
    ranges=[r_mpi, r_msl],
    tuning=Grid(resolution=8),
    resampling=CV(nfolds=5, rng=112),
    operation=predict_mode,
    measure=misclassification_rate
)
mtm = machine(tm, X, y)
fit!(mtm, rows=train)

ypred = predict_mode(mtm, rows=test)
misclassification_rate(ypred, y[test])

We can inspect the parameters of the best model

In [ ]:
fitted_params(mtm).best_model.decision_tree_classifier

‎
@@
@@dropdown
### Decision Tree Regressor
@@
@@dropdown-content

In [ ]:
DTR = @load DecisionTreeRegressor pkg=DecisionTree

boston = dataset("MASS", "Boston")

y, X = unpack(boston, ==(:MedV))

train, test = partition(eachindex(y), 0.5, shuffle=true, rng=551);

scitype(X)

Let's recode the Count as Continuous and then fit a DTR

In [ ]:
X = coerce(X, autotype(X, rules=(:discrete_to_continuous,)))

dtr_model = DTR()
dtr = machine(dtr_model, X, y)

fit!(dtr, rows=train)

ypred = MLJ.predict(dtr, rows=test)
round(rms(ypred, y[test]), sigdigits=3)

Again we can try tuning this a bit, since it's the same idea as before, let's just try to adjust the depth of the tree:

In [ ]:
r_depth = range(dtr_model, :max_depth, lower=2, upper=20)

tm = TunedModel(
    model=dtr_model,
    ranges=[r_depth],
    tuning=Grid(resolution=10),
    resampling=CV(nfolds=5, rng=1254),
    measure=rms
)

mtm = machine(tm, X, y)

fit!(mtm, rows=train)

ypred = MLJ.predict(mtm, rows=test)
round(rms(ypred, y[test]), sigdigits=3)

‎
@@

‎
@@
@@dropdown
## Random Forest
@@
@@dropdown-content

**Note**: the package [`DecisionTree.jl`](https://github.com/bensadeghi/DecisionTree.jl) also has a RandomForest model but it is not yet interfaced with in MLJ.

In [ ]:
RFR = @load RandomForestRegressor pkg=MLJScikitLearnInterface

rf_mdl = RFR()
rf = machine(rf_mdl, X, y)
fit!(rf, rows=train)

ypred = MLJ.predict(rf, rows=test)
round(rms(ypred, y[test]), sigdigits=3)

‎
@@
@@dropdown
## Gradient Boosting Machine
@@
@@dropdown-content

In [ ]:
XGBR = @load XGBoostRegressor

xgb_mdl = XGBR(num_round=10, max_depth=10)
xgb = machine(xgb_mdl, X, y)
fit!(xgb, rows=train)

ypred = MLJ.predict(xgb, rows=test)
round(rms(ypred, y[test]), sigdigits=3)

Again we could do some tuning for this.

‎
@@

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*